## This is a Cyprus LV Strovolides Plots Substation

<img src="Single-Diagram.png">


In [8]:
### Libraries

import pandapower as pp #import pandapower
import pandas as pd
import math
import random
import pandapower.networks as nw
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy import stats
import warnings
import statsmodels.api as sm
from scipy.stats import truncnorm
from datetime import datetime

In [9]:
# Create an empty network
net = pp.create_empty_network(sn_mva=0.315) #create an empty network

### Buses

In [ ]:
# bus1 = pp.create_bus(net, name="HV Busbar", vn_kv=110, type="b")
min_vm_pu=0.95
max_vm_pu=1.05
Feeder1={}
Feeder2={}
Feeder3={}

# GridBus = pp.create_bus(net, vn_kv=11, name="Main-Bus",min_vm_pu=min_vm_pu, max_vm_pu=max_vm_pu, type="b")
FeederBus = pp.create_bus(net, vn_kv=0.4, name="Feeders-Bus",min_vm_pu=min_vm_pu, max_vm_pu=max_vm_pu, type="b")

for i in range(0, 24):
    bus_id = str(i)
    vn_kv = 0.4  # Voltage level from Matpower bus data
    bus_name = f"F1-{bus_id}"
    Feeder1[i]=pp.create_bus(net, vn_kv=vn_kv, name=bus_name,min_vm_pu=min_vm_pu, max_vm_pu=max_vm_pu, type="b")
    
for i in range(0, 8):
    bus_id = str(i)
    vn_kv = 0.4  # Voltage level from Matpower bus data
    bus_name = f"F2-{bus_id}"
    Feeder2[i]=pp.create_bus(net, vn_kv=vn_kv, name=bus_name,min_vm_pu=min_vm_pu, max_vm_pu=max_vm_pu, type="b")

for i in range(0, 29):
    bus_id = str(i)
    vn_kv = 0.4  # Voltage level from Matpower bus data
    bus_name = f"F3-{bus_id}"
    Feeder3[i]=pp.create_bus(net, vn_kv=vn_kv, name=bus_name,min_vm_pu=min_vm_pu, max_vm_pu=max_vm_pu, type="b")
    
# net.bus # show bus table


### External Grid

In [11]:
pp.create_ext_grid(net,FeederBus , vm_pu=1.0, va_degree=0, s_sc_max_mva=10000, rx_max=0.1) # Create an external grid connection

# net.ext_grid["r0x0_max"] = 0.1
# net.ext_grid["x0x_max"] = 1.0

# net.ext_grid #show external grid table

0

### Transformer 

In [12]:
# # trafo1 = pp.create_transformer(net, bus[1], bus[2], name="11kV/0.4kV transformer", std_type="0.63 MVA 10/0.4 kV")
# # net.trafo #show transformer table

# # net.trafo['vn_hv_kv'][0]=11
# # net.trafo['shift_degree'][0]=330

# trafo1 = pp.create_transformer_from_parameters(net, GridBus, lv_bus=FeederBus, 
#                                           sn_mva= 0.315, vn_hv_kv= 11, 
#                                           vn_lv_kv= 0.4, vk_percent= 6, 
#                                           vkr_percent= 0.78125, pfe_kw= 2.7, 
#                                           i0_percent= 0.16875, shift_degree= 330, 
#                                           tap_side= 'lv',tap_neutral= 0,
#                                           tap_min= -2, tap_max= 2,
#                                           tap_step_degree= 0,
#                                           tap_step_percent= 2.5,
#                                           tap_phase_shifter= False,
#                                           vk0_percent= 3, vkr0_percent= 0.78125, 
#                                           mag0_percent= 100, mag0_rx= 0.,
#                                           si0_hv_partial= 0.9,vector_group= "Dyn",
#                                           parallel=1,tap_pos=0,
#                                           index=pp.get_free_id(net.trafo)+1,
#                                            )

# # print(net.trafo)




### Lines

In [ ]:
#### LV 300c AI 31/2-C
l1_r_ohm_per_km = 0.0958   # Resistance per km
l1_x_ohm_per_km = 0.07  # Reactance per km
l1_c_nf_per_km = 0             # Charging susceptance (converted to nF/km if needed)
l1_r0_ohm_per_km = 0.34838   # Resistance per km
l1_x0_ohm_per_km = 1.2328  # Reactance per km
l1_c0_ohm_per_km = 0             # Charging susceptance (converted to nF/km if needed)
l1_max_i_ka = 0.42     # Maximum thermal current rating
l1_length_km = 0.007 

### LV 100mm AI O/H 

l2_r_ohm_per_km = 0.270922   # Resistance per km
l2_x_ohm_per_km = 0.257383  # Reactance per km
l2_c_nf_per_km = 0             # Charging susceptance (converted to nF/km if needed)
l2_r0_ohm_per_km = 0.554544   # Resistance per km
l2_x0_ohm_per_km = 1.030687  # Reactance per km
l2_c0_nf_per_km = 0             # Charging susceptance (converted to nF/km if needed)
l2_max_i_ka = 0.271     # Maximum thermal current rating
l2_length_km = 0.05
l2_length_km_2=0.035
 
### Service Line 

l3_r_ohm_per_km = 1.273   # Resistance per km
l3_x_ohm_per_km = 0.00001  # Reactance per km
l3_c_nf_per_km = 0             # Charging susceptance (converted to nF/km if needed)
l3_r0_ohm_per_km = 1.273   # Resistance per km
l3_x0_ohm_per_km = 0.00001  # Reactance per km
l3_c0_nf_per_km = 0             # Charging susceptance (converted to nF/km if needed)
l3_max_i_ka = 0.099     # Maximum thermal current rating
l3_length_km = 0.02  # Using this i will make the p.u to ohms. 


### Main Feeder Bus to Feeder Start Connection ###

pp.create_line_from_parameters(net, FeederBus, Feeder1[0], l1_length_km, l1_r_ohm_per_km, l1_x_ohm_per_km, l1_c_nf_per_km, l1_max_i_ka,
                    r0_ohm_per_km= l1_r0_ohm_per_km, x0_ohm_per_km= l1_x0_ohm_per_km, c0_nf_per_km=  l1_c0_ohm_per_km )

pp.create_line_from_parameters(net, FeederBus, Feeder2[0], l1_length_km, l1_r_ohm_per_km, l1_x_ohm_per_km, l1_c_nf_per_km, l1_max_i_ka,
                    r0_ohm_per_km= l1_r0_ohm_per_km, x0_ohm_per_km= l1_x0_ohm_per_km, c0_nf_per_km=  l1_c0_ohm_per_km )

pp.create_line_from_parameters(net, FeederBus, Feeder3[0], l1_length_km, l1_r_ohm_per_km, l1_x_ohm_per_km, l1_c_nf_per_km, l1_max_i_ka,
                    r0_ohm_per_km= l1_r0_ohm_per_km, x0_ohm_per_km= l1_x0_ohm_per_km, c0_nf_per_km=  l1_c0_ohm_per_km )

# Feeder 1 
#cross
pp.create_line_from_parameters(net, Feeder1[0], Feeder1[1], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
#cross
pp.create_line_from_parameters(net, Feeder1[1], Feeder1[2], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder1[2], Feeder1[3], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder1[3], Feeder1[4], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
#cross
pp.create_line_from_parameters(net, Feeder1[1], Feeder1[5], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder1[5], Feeder1[6], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder1[6], Feeder1[7], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
#cross
pp.create_line_from_parameters(net, Feeder1[0], Feeder1[8], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder1[8], Feeder1[9], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder1[9], Feeder1[10], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder1[10], Feeder1[11], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
#cross
pp.create_line_from_parameters(net, Feeder1[11], Feeder1[12], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder1[12], Feeder1[13], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder1[13], Feeder1[14], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
#Cross
pp.create_line_from_parameters(net, Feeder1[11], Feeder1[15], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder1[15], Feeder1[16], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder1[16], Feeder1[17], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder1[17], Feeder1[18], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
#cross
pp.create_line_from_parameters(net, Feeder1[18], Feeder1[19], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder1[19], Feeder1[20], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
#cros
pp.create_line_from_parameters(net, Feeder1[18], Feeder1[21], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder1[21], Feeder1[22], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder1[22], Feeder1[23], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )


## Feeder 2
pp.create_line_from_parameters(net, Feeder2[0], Feeder2[1], l2_length_km, l1_r_ohm_per_km, l1_x_ohm_per_km, l1_c_nf_per_km, l1_max_i_ka,
                    r0_ohm_per_km= l1_r0_ohm_per_km, x0_ohm_per_km= l1_x0_ohm_per_km, c0_nf_per_km=  l1_c0_ohm_per_km )

pp.create_line_from_parameters(net, Feeder2[1], Feeder2[2], l2_length_km, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder2[2], Feeder2[3], l2_length_km, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder2[3], Feeder2[4], l2_length_km, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
 
pp.create_line_from_parameters(net, Feeder2[4], Feeder2[5], l2_length_km, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder2[5], Feeder2[6], l2_length_km, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder2[6], Feeder2[7], l2_length_km, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

# Feeder 3
#cros
pp.create_line_from_parameters(net, Feeder3[0], Feeder3[1], l2_length_km_2, l1_r_ohm_per_km, l1_x_ohm_per_km, l1_c_nf_per_km, l1_max_i_ka,
                    r0_ohm_per_km= l1_r0_ohm_per_km, x0_ohm_per_km= l1_x0_ohm_per_km, c0_nf_per_km=  l1_c0_ohm_per_km )
#cros
pp.create_line_from_parameters(net, Feeder3[0], Feeder3[2], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder3[2], Feeder3[3], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
 
pp.create_line_from_parameters(net, Feeder3[3], Feeder3[4], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
#cros
pp.create_line_from_parameters(net, Feeder3[4], Feeder3[5], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder3[5], Feeder3[6], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
#cros
pp.create_line_from_parameters(net, Feeder3[6], Feeder3[7], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
#cros
pp.create_line_from_parameters(net, Feeder3[6], Feeder3[8], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
# #cros
pp.create_line_from_parameters(net, Feeder3[4], Feeder3[9], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder3[9], Feeder3[10], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder3[10], Feeder3[11], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder3[11], Feeder3[12], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
#cros
pp.create_line_from_parameters(net, Feeder3[12], Feeder3[13], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder3[13], Feeder3[14], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
#cros
pp.create_line_from_parameters(net, Feeder3[14], Feeder3[15], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
#cros
pp.create_line_from_parameters(net, Feeder3[14], Feeder3[16], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder3[16], Feeder3[17], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder3[17], Feeder3[18], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder3[12], Feeder3[19], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder3[19], Feeder3[20], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
#cros
pp.create_line_from_parameters(net, Feeder3[20], Feeder3[21], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder3[21], Feeder3[22], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder3[22], Feeder3[23], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
#cros
pp.create_line_from_parameters(net, Feeder3[20], Feeder3[24], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder3[24], Feeder3[25], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
#cros
pp.create_line_from_parameters(net, Feeder3[25], Feeder3[26], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )
#cros
pp.create_line_from_parameters(net, Feeder3[25], Feeder3[27], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

pp.create_line_from_parameters(net, Feeder3[27], Feeder3[28], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )

# net.line # show line table

23

In [14]:
# min_vm_pu=0.95
# max_vm_pu=1.05
# Feeder1_load_bus={}
# Feeder2_load_bus={}
# Feeder3_load_bus={}

# for i in range(0, 24):
#     for j in range (0,2):
#         bus_id = str(i)
#         load_id = str(j)
#         vn_kv = 0.4  # Voltage level from Matpower bus data
#         bus_name = f"F1-Bus-{bus_id}-Load{load_id}"
#         Feeder1_load_bus[i*2+j]=pp.create_bus(net, vn_kv=vn_kv, name=bus_name,min_vm_pu=min_vm_pu, max_vm_pu=max_vm_pu, type="b")
#         pp.create_line_from_parameters(net, Feeder1[i], Feeder1_load_bus[i*2+j], l3_length_km, l3_r_ohm_per_km, l3_x_ohm_per_km, l3_c_nf_per_km, l3_max_i_ka,
#                     r0_ohm_per_km= l3_r0_ohm_per_km, x0_ohm_per_km= l3_x0_ohm_per_km, c0_nf_per_km=  l3_c0_nf_per_km )
# for i in range(0, 8):
#     for j in range (0,5):   
#         bus_id = str(i)
#         load_id = str(j)
#         vn_kv = 0.4  # Voltage level from Matpower bus data
#         bus_name = f"F2-Bus-{bus_id}-Load{load_id}"
#         Feeder2_load_bus[i*5+j]=pp.create_bus(net, vn_kv=vn_kv, name=bus_name,min_vm_pu=min_vm_pu, max_vm_pu=max_vm_pu, type="b")
#         pp.create_line_from_parameters(net, Feeder2[i], Feeder2_load_bus[i*5+j], l3_length_km, l3_r_ohm_per_km, l3_x_ohm_per_km, l3_c_nf_per_km, l3_max_i_ka,
#                     r0_ohm_per_km= l3_r0_ohm_per_km, x0_ohm_per_km= l3_x0_ohm_per_km, c0_nf_per_km=  l3_c0_nf_per_km )
# for i in range(0, 29):
#     for j in range (0,1):
#         bus_id = str(i)
#         load_id = str(j)
#         vn_kv = 0.4  # Voltage level from Matpower bus data
#         bus_name = f"F3-Bus-{bus_id}-Load{load_id}"
#         Feeder3_load_bus[i*1+j]=pp.create_bus(net, vn_kv=vn_kv, name=bus_name,min_vm_pu=min_vm_pu, max_vm_pu=max_vm_pu, type="b")
#         pp.create_line_from_parameters(net, Feeder3[i], Feeder3_load_bus[i*1+j], l3_length_km, l3_r_ohm_per_km, l3_x_ohm_per_km, l3_c_nf_per_km, l3_max_i_ka,
#                     r0_ohm_per_km= l3_r0_ohm_per_km, x0_ohm_per_km= l3_x0_ohm_per_km, c0_nf_per_km=  l3_c0_nf_per_km )

# net.bus # show bus table

#### Loads


In [15]:
# for bus in Feeder1_load_bus:
#     load_name = f"F1-load{bus}"
#     pp.create_asymmetric_load(net, Feeder1_load_bus[bus], p_a_mw=0, q_a_mvar=0, p_b_mw=0, q_b_mvar=0, p_c_mw=0, q_c_mvar=0, name=load_name)
# for bus in Feeder2_load_bus:
#     load_name = f"F2-load{bus}"
#     pp.create_asymmetric_load(net, Feeder2_load_bus[bus], p_a_mw=0, q_a_mvar=0, p_b_mw=0, q_b_mvar=0, p_c_mw=0, q_c_mvar=0, name=load_name)
# for bus in Feeder3_load_bus:
#     load_name = f"F3-load{bus}"
#     pp.create_asymmetric_load(net, Feeder3_load_bus[bus], p_a_mw=0, q_a_mvar=0, p_b_mw=0, q_b_mvar=0, p_c_mw=0, q_c_mvar=0, name=load_name)

In [16]:
# for i,bus in Feeder1.items():
#     load_name = f"F1-load{bus}"
#     pp.create_load(net, Feeder1_load_bus[i], p_mw=10/1000, q_mvar=10*math.tan(math.acos(0.95))/1000, name=load_name)

# for i,bus in Feeder2_load_bus.items():
#     load_name = f"F2-load{bus}"
#     pp.create_load(net, Feeder2_load_bus[i], p_mw=10/1000, q_mvar=10*math.tan(math.acos(0.95))/1000, name=load_name)

# for i,bus in Feeder3_load_bus.items():
#     load_name = f"F3-load{bus}"
#     pp.create_load(net, Feeder3_load_bus[i], p_mw=10/1000, q_mvar=10*math.tan(math.acos(0.95))/1000, name=load_name)


In [ ]:
for i,bus in Feeder1.items():
    load_name = f"F1-load{bus}"
    pp.create_load(net, Feeder1[i], p_mw=20/1000, q_mvar=20*math.tan(math.acos(0.95))/1000, name=load_name)

for i,bus in Feeder2.items():
    load_name = f"F2-load{bus}"
    pp.create_load(net, Feeder2[i], p_mw=20/1000, q_mvar=20*math.tan(math.acos(0.95))/1000, name=load_name)

for i,bus in Feeder3.items():
    load_name = f"F3-load{bus}"
    pp.create_load(net, Feeder3[i], p_mw=10/1000, q_mvar=20*math.tan(math.acos(0.95))/1000, name=load_name)
    

In [18]:
########## Custom Create DER ##############

########## Feeder 1 ##############
# sgen1 = pp.create_sgen(net, Feeder1_load_bus[0], p_mw=7/1000, q_mvar=-math.tan(math.acos(0.95))*7/1000, name="DER 1")
# sgen2 = pp.create_sgen(net, Feeder1_load_bus[4], p_mw=6/1000, q_mvar=-math.tan(math.acos(0.95))*6/1000, name="DER 2")
# sgen3 = pp.create_sgen(net, Feeder1_load_bus[8], p_mw=5/1000, q_mvar=-math.tan(math.acos(0.95))*5/1000, name="DER 3")
# sgen4 = pp.create_sgen(net, Feeder1_load_bus[12], p_mw=4/1000, q_mvar=-math.tan(math.acos(0.95))*4/1000, name="DER 4")
# sgen5 = pp.create_sgen(net, Feeder1_load_bus[16], p_mw=7/1000, q_mvar=-math.tan(math.acos(0.95))*7/1000, name="DER 5")
# sgen6 = pp.create_sgen(net, Feeder1_load_bus[20], p_mw=6/1000, q_mvar=-math.tan(math.acos(0.95))*6/1000, name="DER 6")
# sgen7 = pp.create_sgen(net, Feeder1_load_bus[24], p_mw=5/1000, q_mvar=-math.tan(math.acos(0.95))*5/1000, name="DER 7")
# sgen8 = pp.create_sgen(net, Feeder1_load_bus[28], p_mw=4/1000, q_mvar=-math.tan(math.acos(0.95))*4/1000, name="DER 8")
# sgen9 = pp.create_sgen(net, Feeder1_load_bus[32], p_mw=7/1000, q_mvar=-math.tan(math.acos(0.95))*7/1000, name="DER 9")
# sgen10 = pp.create_sgen(net, Feeder1_load_bus[36], p_mw=6/1000, q_mvar=-math.tan(math.acos(0.95))*6/1000, name="DER 10")
# sgen11 = pp.create_sgen(net, Feeder1_load_bus[40], p_mw=5/1000, q_mvar=-math.tan(math.acos(0.95))*5/1000, name="DER 11")
# sgen12 = pp.create_sgen(net, Feeder1_load_bus[44], p_mw=4/1000, q_mvar=-math.tan(math.acos(0.95))*4/1000, name="DER 12")
# sgen13 = pp.create_sgen(net, Feeder1_load_bus[21], p_mw=8/1000, q_mvar=-math.tan(math.acos(0.95))*8/1000, name="DER 13")
# sgen14 = pp.create_sgen(net, Feeder1_load_bus[38], p_mw=10/1000, q_mvar=-math.tan(math.acos(0.95))*10/1000, name="DER 14")


########## Feeder 2 ##############
# sgen15 = pp.create_sgen(net, Feeder2_load_bus[0], p_mw=5/1000, q_mvar=-math.tan(math.acos(0.95))*5/1000, name="DER 15")
# sgen16 = pp.create_sgen(net, Feeder2_load_bus[4], p_mw=4/1000, q_mvar=-math.tan(math.acos(0.95))*4/1000, name="DER 16")
# sgen17 = pp.create_sgen(net, Feeder2_load_bus[8], p_mw=7/1000, q_mvar=-math.tan(math.acos(0.95))*7/1000, name="DER 17")
# sgen18 = pp.create_sgen(net, Feeder2_load_bus[12], p_mw=6/1000, q_mvar=-math.tan(math.acos(0.95))*6/1000, name="DER 18")
# sgen19 = pp.create_sgen(net, Feeder2_load_bus[16], p_mw=10/1000, q_mvar=-math.tan(math.acos(0.95))*10/1000, name="DER 19")
# sgen20 = pp.create_sgen(net, Feeder2_load_bus[20], p_mw=4/1000, q_mvar=-math.tan(math.acos(0.95))*4/1000, name="DER 20")
# sgen21 = pp.create_sgen(net, Feeder2_load_bus[22], p_mw=5/1000, q_mvar=-math.tan(math.acos(0.95))*5/1000, name="DER 21")
# sgen22 = pp.create_sgen(net, Feeder2_load_bus[24], p_mw=9/1000, q_mvar=-math.tan(math.acos(0.95))*9/1000, name="DER 22")
# sgen23 = pp.create_sgen(net, Feeder2_load_bus[28], p_mw=7/1000, q_mvar=-math.tan(math.acos(0.95))*7/1000, name="DER 23")
# sgen24 = pp.create_sgen(net, Feeder2_load_bus[32], p_mw=6/1000, q_mvar=-math.tan(math.acos(0.95))*6/1000, name="DER 24")
# sgen25 = pp.create_sgen(net, Feeder2_load_bus[36], p_mw=10/1000, q_mvar=-math.tan(math.acos(0.95))*10/1000, name="DER 25")
# sgen26 = pp.create_sgen(net, Feeder2_load_bus[39], p_mw=4/1000, q_mvar=-math.tan(math.acos(0.95))*4/1000, name="DER 26")

########## Feeder 3 ##############

# sgen27 = pp.create_sgen(net, Feeder3_load_bus[0], p_mw=5/1000, q_mvar=-math.tan(math.acos(0.95))*5/1000, name="DER 27")
# sgen28 = pp.create_sgen(net, Feeder3_load_bus[4], p_mw=4/1000, q_mvar=-math.tan(math.acos(0.95))*4/1000, name="DER 28")
# sgen29 = pp.create_sgen(net, Feeder3_load_bus[8], p_mw=7/1000, q_mvar=-math.tan(math.acos(0.95))*7/1000, name="DER 29")
# sgen30 = pp.create_sgen(net, Feeder3_load_bus[12], p_mw=6/1000, q_mvar=-math.tan(math.acos(0.95))*6/1000, name="DER 30")
# sgen31 = pp.create_sgen(net, Feeder3_load_bus[16], p_mw=5/1000, q_mvar=-math.tan(math.acos(0.95))*5/1000, name="DER 31")
# sgen32 = pp.create_sgen(net, Feeder3_load_bus[20], p_mw=4/1000, q_mvar=-math.tan(math.acos(0.95))*4/1000, name="DER 32")
# sgen33 = pp.create_sgen(net, Feeder3_load_bus[24], p_mw=9/1000, q_mvar=-math.tan(math.acos(0.95))*9/1000, name="DER 33")
# sgen34 = pp.create_sgen(net, Feeder3_load_bus[28], p_mw=6/1000, q_mvar=-math.tan(math.acos(0.95))*6/1000, name="DER 34")
# sgen35 = pp.create_sgen(net, Feeder3_load_bus[10], p_mw=10/1000, q_mvar=-math.tan(math.acos(0.95))*10/1000, name="DER 35")

In [ ]:
########## Custom Create DER ##############

########## Feeder 1 ##############
sgen1 = pp.create_sgen(net, Feeder1[0], p_mw=7/1000, q_mvar=-math.tan(math.acos(0.95))*7/1000, name="DER 1")
sgen2 = pp.create_sgen(net, Feeder1[2], p_mw=6/1000, q_mvar=-math.tan(math.acos(0.95))*6/1000, name="DER 2")
sgen3 = pp.create_sgen(net, Feeder1[4], p_mw=5/1000, q_mvar=-math.tan(math.acos(0.95))*5/1000, name="DER 3")
sgen4 = pp.create_sgen(net, Feeder1[6], p_mw=4/1000, q_mvar=-math.tan(math.acos(0.95))*4/1000, name="DER 4")
sgen5 = pp.create_sgen(net, Feeder1[8], p_mw=7/1000, q_mvar=-math.tan(math.acos(0.95))*7/1000, name="DER 5")
sgen6 = pp.create_sgen(net, Feeder1[10], p_mw=6/1000, q_mvar=-math.tan(math.acos(0.95))*6/1000, name="DER 6")
sgen7 = pp.create_sgen(net, Feeder1[12], p_mw=5/1000, q_mvar=-math.tan(math.acos(0.95))*5/1000, name="DER 7")
sgen8 = pp.create_sgen(net, Feeder1[14], p_mw=4/1000, q_mvar=-math.tan(math.acos(0.95))*4/1000, name="DER 8")
sgen9 = pp.create_sgen(net, Feeder1[16], p_mw=7/1000, q_mvar=-math.tan(math.acos(0.95))*7/1000, name="DER 9")
sgen10 = pp.create_sgen(net, Feeder1[18], p_mw=6/1000, q_mvar=-math.tan(math.acos(0.95))*6/1000, name="DER 10")
sgen11 = pp.create_sgen(net, Feeder1[20], p_mw=5/1000, q_mvar=-math.tan(math.acos(0.95))*5/1000, name="DER 11")
sgen12 = pp.create_sgen(net, Feeder1[22], p_mw=4/1000, q_mvar=-math.tan(math.acos(0.95))*4/1000, name="DER 12")

sgen13 = pp.create_sgen(net, Feeder2[0], p_mw=7/1000, q_mvar=-math.tan(math.acos(0.95))*7/1000, name="DER 13")
sgen14 = pp.create_sgen(net, Feeder2[2], p_mw=6/1000, q_mvar=-math.tan(math.acos(0.95))*6/1000, name="DER 14")
sgen15 = pp.create_sgen(net, Feeder2[4], p_mw=5/1000, q_mvar=-math.tan(math.acos(0.95))*5/1000, name="DER 15")
sgen16 = pp.create_sgen(net, Feeder2[6], p_mw=4/1000, q_mvar=-math.tan(math.acos(0.95))*4/1000, name="DER 16")
sgen17 = pp.create_sgen(net, Feeder2[7], p_mw=7/1000, q_mvar=-math.tan(math.acos(0.95))*7/1000, name="DER 17")

sgen18 = pp.create_sgen(net, Feeder3[0], p_mw=7/1000, q_mvar=-math.tan(math.acos(0.95))*7/1000, name="DER 18")
sgen19 = pp.create_sgen(net, Feeder3[2], p_mw=6/1000, q_mvar=-math.tan(math.acos(0.95))*6/1000, name="DER 19")
sgen20 = pp.create_sgen(net, Feeder3[4], p_mw=5/1000, q_mvar=-math.tan(math.acos(0.95))*5/1000, name="DER 20")
sgen21 = pp.create_sgen(net, Feeder3[6], p_mw=4/1000, q_mvar=-math.tan(math.acos(0.95))*4/1000, name="DER 21")
sgen22 = pp.create_sgen(net, Feeder3[7], p_mw=7/1000, q_mvar=-math.tan(math.acos(0.95))*7/1000, name="DER 22")

In [20]:
# # add any necessary zero_impedance parameters
# pp.add_zero_impedance_parameters(net)